In [11]:
%reset
import pandas as pd
from yelpapi import YelpAPI
import requests
import os
API_KEY = 'YOUR API KEY'
yelp_api = YelpAPI(API_KEY)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [12]:
# Function to take in latitude longitude coordinates and return restaurants
def get_businesses(latitude, longitude, term, api_key):
    headers = {'Authorization': 'Bearer %s' % API_KEY}
    url = 'https://api.yelp.com/v3/businesses/search'

    data = []
    
    # setting the offset iteration is really important here for Yelp's requirements allow us to pull 50 rows at a time and up to 1000 max rows for the parameters that are set
    for offset in range(0, 1000, 50):
        # set the parameters
        params = {
            'limit': 50,
            'latitude': latitude,
            'longitude': longitude,
            #'location': location.replace(' ', '+'),
            'term': term.replace(' ', '+'),
            # first iteration will return 1-50, second will return 51-100, and so on until 951-1000
            'offset': offset,
            # radius can be adjusted to your preference 
            'radius': 750,
            # this sort_by parameter only pulls restaurants based on distance. See documentation for other preferences like best match
            'sort_by': 'distance'
}
        # use requests library to pull data from Yelp API
        response = requests.get(url, headers=headers, params=params)
        # check if response was good (200) or bad (400)
        if response.status_code == 200:
            # append new data to last iteration
            data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break
    return data

In [13]:
# term can be changed based on what the API request should be 
term = 'restaurant'

# read in coordinates data set to get latitude and longitude coordinates for the function 
coordinates = pd.read_csv("~/Desktop/Coordinates.csv")

latitude = coordinates['latitude'].astype(str)
longitude = coordinates['longitude'].astype(str)

In [14]:
import datetime
# start time of running function
StartTime = datetime.datetime.now()

data=[]
# set maxReturn = {1 if all 1000 restaurants returned, 0 otherwise}, this can help to adjust radius, if there a lot of 1's then there is likely uneccesary overlap slowing run time
maxReturn = [0]*len(latitude)

# running a loop on our coordinates with get_businesses function (could be included in the function above as another function but also makes sense here)
for i in range(0,len(latitude)):
    # record each response
    response = get_businesses(latitude[i], longitude[i], term, API_KEY)
    # append data with each new response
    data += response
    # record maxReturn
    if len(response) == 1000:
        maxReturn[i] = 1
        
# get elapsed time
EndTime = datetime.datetime.now()
print(EndTime - StartTime)

0:59:07.941989


In [15]:
# turn data into a dataframe, drop duplicates (from potential overlap), reset index, show final df
df = pd.DataFrame(data)
df = df.drop_duplicates(subset='id', keep="first")
df.reset_index(drop=True, inplace=True)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,tImw3a1oveSqMSjQBQq9TA,juice-healthy-food-and-drink-new-york-2,Juice Healthy Food & Drink,https://s3-media3.fl.yelpcdn.com/bphoto/GDqhtd...,False,https://www.yelp.com/biz/juice-healthy-food-an...,100,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",4.0,"{'latitude': 40.74525, 'longitude': -73.97937}","[delivery, pickup]",$$,"{'address1': '156 E 33rd St', 'address2': None...",+12128821611,(212) 882-1611,49.062471
1,OKf1e4V3HmpjzMXSlPhHcQ,al-horno-lean-mexican-kitchen-new-york-18,Al Horno Lean Mexican Kitchen,,False,https://www.yelp.com/biz/al-horno-lean-mexican...,1,"[{'alias': 'mexican', 'title': 'Mexican'}]",1.0,"{'latitude': 40.74525, 'longitude': -73.97937}","[delivery, pickup]",NaN,"{'address1': '156 E 33rd St', 'address2': None...",,,49.572204
2,5YV3zSR18X4Ja6d-TOEtyA,pino-wine-bar-new-york,Pino Wine Bar,https://s3-media1.fl.yelpcdn.com/bphoto/phyELJ...,False,https://www.yelp.com/biz/pino-wine-bar-new-yor...,159,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.5,"{'latitude': 40.74525, 'longitude': -73.97937}",[delivery],$$,"{'address1': '156 E 33rd St', 'address2': '', ...",+19179913718,(917) 991-3718,49.572204
3,EOdXbTaHZ0BomaabxDHeoA,dominos-pizza-new-york-8,Domino's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/o7Vv-6...,False,https://www.yelp.com/biz/dominos-pizza-new-yor...,50,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 40.7443016, 'longitude': -73.9793...","[delivery, pickup]",$,"{'address1': '464 3rd Ave', 'address2': '', 'a...",+12125450200,(212) 545-0200,50.202655
4,oGq_6DQ1AoQrAYfFSDV_-Q,mcdonalds-new-york-110,McDonald's,https://s3-media1.fl.yelpcdn.com/bphoto/UYSv-9...,False,https://www.yelp.com/biz/mcdonalds-new-york-11...,35,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.5,"{'latitude': 40.744904, 'longitude': -73.978949}",[delivery],$,"{'address1': '480 3rd Ave', 'address2': '', 'a...",+12125328186,(212) 532-8186,53.266732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15114,aKXL5kyajpeI0MffXDWLiw,panda-queen-restaurant-astoria,Panda Queen Restaurant,,False,https://www.yelp.com/biz/panda-queen-restauran...,1,"[{'alias': 'restaurants', 'title': 'Restaurant...",3.0,"{'latitude': 40.76836, 'longitude': -73.911}",[],$,"{'address1': '2421 Steinway St', 'address2': '...",+17185459655,(718) 545-9655,452.926844
15115,ZfVHyRKrP7OQnaUmItRK1A,duzan-mediterranean-grill-astoria,Duzan Mediterranean Grill,https://s3-media4.fl.yelpcdn.com/bphoto/3QN_Mb...,False,https://www.yelp.com/biz/duzan-mediterranean-g...,589,"[{'alias': 'mideastern', 'title': 'Middle East...",4.0,"{'latitude': 40.7684923638382, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '2411 Steinway St', 'address2': N...",+17182047488,(718) 204-7488,458.480023
15116,UqYxtrutXYdielZXMFLZug,layali-dubai-astoria,Layali Dubai,https://s3-media2.fl.yelpcdn.com/bphoto/cCMjoL...,False,https://www.yelp.com/biz/layali-dubai-astoria?...,137,"[{'alias': 'mideastern', 'title': 'Middle East...",3.5,"{'latitude': 40.7685091, 'longitude': -73.9109...","[pickup, delivery]",$$,"{'address1': '24-17 Steinway St', 'address2': ...",+17187281492,(718) 728-1492,467.925125
15117,jDq9PWUwrSdJ_k1tHUnKFg,marina-seafood-and-restaurant-astoria,Marina Seafood & Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/ye-OW8...,False,https://www.yelp.com/biz/marina-seafood-and-re...,7,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",5.0,"{'latitude': 40.7685223, 'longitude': -73.9109...","[pickup, delivery]",NaN,"{'address1': '2415 Steinway St', 'address2': '...",,,474.328470


In [ ]:
# write to csv, pandas has a lot of writing options df.to_excel() for example would write to an Excel sheet
df.to_csv("~/Desktop/YelpAPIResults.csv")